In [1]:
from BaseModel import SentenceEmbedder, MeanPooling, Normalize
import torch
import torch.nn as nn
import pytorch_lightning as pl
from losses.Softmax import SoftmaxTrainer
from transformers import AutoTokenizer

In [2]:
model_1 = SoftmaxTrainer(
    pretrained_model= "sentence-transformers/all-MiniLM-L6-v2",
    pooling_layer= MeanPooling(),
    normalize_layer= Normalize(),
)

/home/carles/undergrad-thesis/.venv/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'pooling_layer' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['pooling_layer'])`.
  rank_zero_warn(
/home/carles/undergrad-thesis/.venv/lib/python3.8/site-packages/pytorch_lightning/utilities/parsing.py:196: UserWarning: Attribute 'normalize_layer' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['normalize_layer'])`.
  rank_zero_warn(


In [3]:
#test take input
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
sentence_1 = ["I like to eat apples."]
input = tokenizer(sentence_1, return_tensors="pt", padding="max_length", truncation=True, max_length=512)

In [4]:
input

{'input_ids': tensor([[  101,  1045,  2066,  2000,  4521, 18108,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [5]:
model_1(input)

tensor([[ 2.0287e-02, -1.0149e-02,  4.4379e-03,  9.8936e-02, -7.5347e-02,
         -3.1618e-02,  1.2500e-01, -7.0418e-02,  3.1395e-02,  1.3503e-02,
          4.7869e-02, -1.1311e-01,  2.2448e-02,  4.3147e-03,  4.1479e-02,
         -3.1775e-02,  6.4389e-02, -3.9002e-03, -2.0846e-02,  2.3466e-02,
         -7.8653e-02,  6.1797e-02,  5.7146e-02, -2.2043e-02, -8.3824e-03,
          5.8999e-02,  5.5665e-02, -3.1469e-02, -9.1467e-02, -1.2344e-02,
         -6.5881e-02, -3.4765e-03,  1.1020e-02,  2.4454e-02, -3.5012e-02,
         -6.7719e-03,  1.3620e-01, -7.4847e-02, -3.0549e-02, -6.1541e-03,
          3.0408e-02,  5.5383e-02,  5.7821e-02,  3.7068e-02, -7.2129e-02,
         -7.6281e-03,  7.6650e-03, -1.8241e-02,  1.0269e-01,  4.7048e-02,
          3.0462e-02, -2.0631e-03, -3.7888e-02, -4.1398e-02,  5.3176e-02,
          5.4241e-02,  1.4381e-02,  1.1565e-02,  1.3236e-02,  1.0639e-02,
          6.3619e-02, -6.9950e-02, -1.4093e-03,  2.6537e-02,  3.2536e-02,
         -6.5808e-02, -6.1245e-02,  1.

In [19]:
model_1(input)

TypeError: forward() got an unexpected keyword argument 'input_ids'

In [6]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class TestDataset(Dataset):
    def __init__(self, tokenizer, sentences, labels):
        self.tokenizer = tokenizer
        
        self.labels = labels

In [22]:
from datasets import load_dataset
dataset = load_dataset("carlesoctav/en-id-parallel-sentences")

Found cached dataset parquet (/home/carles/.cache/huggingface/datasets/carlesoctav___parquet/carlesoctav--en-id-parallel-sentences-2ac6d941a9b892f7/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/8 [00:00<?, ?it/s]

In [23]:
# convert to dataset
dataset = dataset["QED"]

In [24]:
dataset.set_format(type='torch', columns=['text_en', 'text_id'])

In [30]:
#tokenize
dataset = dataset.map(lambda x: tokenizer(x["text_en"], x["text_id"], padding="max_length", truncation=True, max_length=512), batched=True,num_proc=96)

Map (num_proc=96):   0%|          | 0/274581 [00:00<?, ? examples/s]

In [32]:
from torch.utils.data import DataLoader
data = DataLoader(dataset, batch_size=32, num_workers=96)

In [34]:
for batch in data:
    print(batch)
    break

{'text_en': ['In the time of Papuan independence, the Papuan flag and the Dutch flag raised together The transition took place when UN came and the Dutch left Papua. The Dutch flag was brought down, only the Papuan and UN flag were flying', 'So we knew that the Ducth was hesitant, the dutch wanted to grant the independence for the people in Papua. But the Papua people cannot acknowledge this secret. Then the Dutch is forced to give the Independence to the Papuan', "The Dutch gave the Independence to Papuan but that was not aired on Radio Republic of Indonesia that is why i have been given a mandate to read the PEPERA text that is crucial in determining Papuan's destiny in a month i was trained to read day and night as I read the PEPERA text there were no mistake allowed because the text is handwriting on 14th July 1969, i had to read it", 'The illeterate people in Papua were only given the stamp by the leader of the village', 'we have accutomed to rule the country with the tradition th